<a href="https://colab.research.google.com/github/DilshanBotheju/BankMarketingPredictionSystem/blob/main/ModelDevelopment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
import numpy as np
import pandas as pd

In [117]:
# Load the data set with seperated columns
data_path = "/content/drive/MyDrive/CM2604_CW/bank-additional-full.csv"
bank_data = pd.read_csv(data_path, encoding = "latin-1", sep =";")
bank_data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [118]:
# To get no. of columns and rows
bank_data.shape

(41188, 21)

In [119]:
# To identify null values
bank_data.isnull().sum()

,0
age,0
job,0
marital,0
education,0
default,0
housing,0
loan,0
contact,0
month,0
day_of_week,0


In [120]:
# Replace places with unknown to null in whole dataset
bank_data = bank_data.replace("unknown",pd.NA)

In [121]:
# To calculate no. of null fields in each column
bank_data.isnull().sum()

,0
age,0
job,330
marital,80
education,1731
default,8597
housing,990
loan,990
contact,0
month,0
day_of_week,0


In [122]:
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             40858 non-null  object 
 2   marital         41108 non-null  object 
 3   education       39457 non-null  object 
 4   default         32591 non-null  object 
 5   housing         40198 non-null  object 
 6   loan            40198 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [123]:
# Getting category counts for y column
bank_data["y"].value_counts()

,count
y,
no,36548
yes,4640


In [124]:
import sklearn
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Label encoding for "y" column
bank_data['y'] = label_encoder.fit_transform(bank_data['y'])

In [125]:
# Convert columns to numerical columns
numeric_bank_data = bank_data.select_dtypes(include= "number")

In [126]:
bank_data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high.school,<NA>,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0


In [127]:
# To identify correlation among y column and other numeric columns
correlation = numeric_bank_data.corr()["y"]
correlation

,y
age,0.030399
duration,0.405274
campaign,-0.066357
pdays,-0.324914
previous,0.230181
emp.var.rate,-0.298334
cons.price.idx,-0.136211
cons.conf.idx,0.054878
euribor3m,-0.307771
nr.employed,-0.354678


Job column

In [128]:
# To handle null values when encoding
def setJob(job):
    if pd.isna(job):
        return "Null"
    else:
        return job

bank_data["job"] = bank_data["job"].apply(setJob)

In [129]:
bank_data["job"].value_counts()

,count
job,
admin.,10422
blue-collar,9254
technician,6743
services,3969
management,2924
retired,1720
entrepreneur,1456
self-employed,1421
housemaid,1060


Marital column

In [130]:
# Handle null values when enoding

def setMarital(marital):
    if pd.isna(marital):
        return "Null"
    else:
        return marital

bank_data["marital"] = bank_data["marital"].apply(setMarital)

In [131]:
bank_data["marital"].value_counts()

,count
marital,
married,24928
single,11568
divorced,4612
Null,80


education column

In [132]:
# Making the available categories simple
def setEducation(education):
    if pd.isna(education):
        return "Null"
    elif education in ["basic.9y", "basic.6y", "basic.4y"]:
        return "basic"
    else:
        return education

# Applying the function
bank_data["education"] = bank_data["education"].apply(setEducation)

In [133]:
bank_data["education"].value_counts()

,count
education,
basic,12513
university.degree,12168
high.school,9515
professional.course,5243
Null,1731
illiterate,18


In [134]:
bank_data.head(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high.school,<NA>,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,admin.,married,basic,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0


default amount column

In [135]:
# Handle null values in default column
def setDefault(default):
    if pd.isna(default):
        return "Null"
    else:
        return default

bank_data["default"] = bank_data["default"].apply(setDefault)

In [136]:
bank_data["default"].value_counts()

,count
default,
no,32588
Null,8597
yes,3


housing loan column

In [137]:
# Handle null values
def setHousing(housing):
    if pd.isna(housing):
        return "Null"
    else:
        return housing

bank_data["housing"] = bank_data["housing"].apply(setHousing)

In [138]:
bank_data["housing"].value_counts()

,count
housing,
yes,21576
no,18622
Null,990


personal loan column

In [139]:
# Handle null values
def setPersonal(personal):
    if pd.isna(personal):
        return "Null"
    else:
        return personal

bank_data["loan"] = bank_data["loan"].apply(setPersonal)

In [140]:
bank_data["loan"].value_counts()

,count
loan,
no,33950
yes,6248
Null,990


pdays column

In [141]:
# Convert 999 days to -1
bank_data["pdays"] = bank_data["pdays"].replace(999,-1)

In [142]:
bank_data["pdays"].value_counts()

,count
pdays,
-1,39673
3,439
6,412
4,118
9,64
2,61
7,60
12,58
10,52


In [143]:
# To see the data types
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [144]:
# Getting categorical columns
categorical_bank_data = bank_data.select_dtypes(include="object")

In [145]:
# Label encoding categorical colums
le = LabelEncoder()

for column in categorical_bank_data.columns:
    bank_data[column] = le.fit_transform(bank_data[column])

In [146]:
# Display encoded dataset
bank_data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,4,2,1,1,1,1,1,6,1,...,1,-1,0,1,1.1,93.994,-36.4,4.857,5191.0,0
1,57,8,2,2,0,1,1,1,6,1,...,1,-1,0,1,1.1,93.994,-36.4,4.857,5191.0,0
2,37,8,2,2,1,2,1,1,6,1,...,1,-1,0,1,1.1,93.994,-36.4,4.857,5191.0,0
3,40,1,2,1,1,1,1,1,6,1,...,1,-1,0,1,1.1,93.994,-36.4,4.857,5191.0,0
4,56,8,2,2,1,1,2,1,6,1,...,1,-1,0,1,1.1,93.994,-36.4,4.857,5191.0,0


In [148]:
from sklearn.preprocessing import StandardScaler

# Assuming numeric_bank_data is a subset of bank_data
numeric_columns = bank_data.select_dtypes(include=['float64', 'int64']).columns

# Standardize the numeric columns
ss = StandardScaler()
bank_data[numeric_columns] = ss.fit_transform(bank_data[numeric_columns])

In [149]:
bank_data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,1.533034,-0.176579,-0.270028,-1.000310,0.513260,-0.915462,-0.319712,1.31827,0.762558,-0.718834,...,-0.565922,-0.170834,-0.349494,0.192622,0.648092,0.722722,0.886447,0.71246,0.33168,-0.356309
1,1.628993,0.947351,-0.270028,-0.429828,-1.946613,-0.915462,-0.319712,1.31827,0.762558,-0.718834,...,-0.565922,-0.170834,-0.349494,0.192622,0.648092,0.722722,0.886447,0.71246,0.33168,-0.356309
2,-0.290186,0.947351,-0.270028,-0.429828,0.513260,0.916174,-0.319712,1.31827,0.762558,-0.718834,...,-0.565922,-0.170834,-0.349494,0.192622,0.648092,0.722722,0.886447,0.71246,0.33168,-0.356309
3,-0.002309,-1.019527,-0.270028,-1.000310,0.513260,-0.915462,-0.319712,1.31827,0.762558,-0.718834,...,-0.565922,-0.170834,-0.349494,0.192622,0.648092,0.722722,0.886447,0.71246,0.33168,-0.356309
4,1.533034,0.947351,-0.270028,-0.429828,0.513260,-0.915462,2.184721,1.31827,0.762558,-0.718834,...,-0.565922,-0.170834,-0.349494,0.192622,0.648092,0.722722,0.886447,0.71246,0.33168,-0.356309


In [150]:
correlation = bank_data.corr()["y"]
correlation

,y
age,0.030399
job,0.025419
marital,0.044538
education,0.048933
default,0.099199
housing,0.011379
loan,-0.003142
contact,-0.144773
month,-0.006065
day_of_week,0.015967
